In [ ]:
from __future__ import print_function, division

%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns; sns.set(context="poster")
import ipywidgets
import yt
import glob
import os
import warnings
import h5py

import numpy as np
import pandas as pd

from astropy import constants as const
from astropy import units as u

M_solar = const.M_sun.cgs.value
m_proton = const.m_p.cgs.value
pc = u.pc.to(u.cm)
yr = u.yr.to(u.s)
Myr = 1e6*yr
gamma = 5/3

@yt.derived_field(name="pressure", units="g  / s**2 / cm")
def _pressure(field, data):
    return (gamma-1) * data["thermal_energy"] * data["density"]

## 1D (hi-res)

In [ ]:
## Boilerplate path hack to give access to full clustered_SNe package
import sys, os
if __package__ is None:
    if os.pardir not in sys.path[0]:
        file_dir = os.getcwd()
        sys.path.insert(0, os.path.join(file_dir, 
                                        os.pardir, 
                                        os.pardir))
        
from clustered_SNe.analysis.parse import Overview, RunSummary

In [ ]:
data_dir_1D = "1D_data/"

run_summary = RunSummary(data_dir_1D, "25451948-485f-46fe-b87b-f4329d03b203")

# 3D (resolution study)

In [ ]:
base_box_size = 400

N_particles_per_side = np.array([
        150, 
#         200, 
#         250, 
        300])
dxs = base_box_size / N_particles_per_side

simulation_base_name = "cluster_cooling"

In [ ]:
possible_SN_files = glob.glob(os.path.join("../ICs/{}_{}".format(simulation_base_name,
                                                                 N_particles_per_side[0])
                                           , "*SNe.dat"))

if len(possible_SN_files) == 0: 
    raise FileNotFoundError("No SN data files found in {}".format(SN_directory))
elif len(possible_SN_files) > 1:
    raise RuntimeError("Too many SN data files found in {}".format(SN_directory))

    
SN_file = possible_SN_files[0]
SN_data = np.loadtxt(SN_file, ndmin=2)
SN_times_3D = np.sort(SN_data[:,0])

SN_times_3D -= SN_times_3D[0]
SN_times_3D[0] = 3e10

SN_times_3D /= u.Myr.to(u.s)

In [ ]:
unit_base = {
    "UnitLength_in_cm" : (pc),
    "UnitVelocity_in_cm_per_s"   : (pc / Myr),
    "UnitMass_in_g"   : (M_solar)
}

#####################

snapshot_filename_format = "snapshot_???.hdf5"

In [ ]:
def total_radial_momentum_from_snapshot_file(snapshot_filename):
    f = h5py.File(snapshot_filename, mode="r")
    masses_shape = f["PartType0"]["Masses"].shape
    new_shape = masses_shape + (1,)

    r_hat = f["PartType0"]["Coordinates"] - (f["Header"].attrs["BoxSize"]/2)
    r_hat = r_hat / (np.sum(r_hat**2, axis=1, dtype=float).reshape(new_shape))**.5
    
    mom = np.sum(r_hat * f["PartType0"]["Velocities"] \
    * np.reshape(f["PartType0"]["Masses"], new_shape), dtype=float)
    
    f.close()
    return mom * M_solar * pc / (1e6*yr)

In [ ]:
def get_times_and_radial_momenta(N_particles_per_side_i):
    snapshot_dir = "../output/{}_{}".format(simulation_base_name, N_particles_per_side_i)
    
    n_files_ready = len(glob.glob(os.path.join(snapshot_dir, snapshot_filename_format)))
    
    if n_files_ready == 0:
#         raise FileNotFoundError("No snapshots found in {}".format(snapshot_dir))
        return np.empty(0), np.empty(0)

    ts = yt.load(os.path.join(snapshot_dir, snapshot_filename_format),
                 unit_base=unit_base)

    times_snapshots_tmp = np.array([ts[j].current_time.convert_to_cgs() 
        for j in range(len(ts))]) / u.Myr.to(u.s)

    print("Loaded {} snapshots".format(len(ts)))

    ds = ts[0]
    rho_0 = ds.all_data()["all","density"].mean()

    
    #########
    snapshot_filenames = sorted(glob.glob(os.path.join(snapshot_dir, snapshot_filename_format)))

    radial_momenta_tmp = np.array([total_radial_momentum_from_snapshot_file(snapshot_filename)
                                   for snapshot_filename in snapshot_filenames])

        
    return times_snapshots_tmp, radial_momenta_tmp

In [ ]:
from multiprocessing import Pool

pool = Pool(2)

outputs = pool.map(get_times_and_radial_momenta, N_particles_per_side)

times_snapshots = {N_particles_per_side_i : outputs[i][0]
                   for i, N_particles_per_side_i in enumerate(N_particles_per_side)}

radial_momenta  = {N_particles_per_side_i : outputs[i][1]
                   for i, N_particles_per_side_i in enumerate(N_particles_per_side)}


In [ ]:
run_summary.momentum / (100 * M_solar * 11 * 1e5)

In [ ]:
with sns.axes_style("ticks"):
    plt.plot((run_summary.times - run_summary.overview.SNe_times[0]) / (1e6*yr),
             run_summary.momentum / (100 * M_solar * 11 * 1e5),
             label="1D",
             linestyle="dashed",
             )


    for N_particles_per_side_i in N_particles_per_side:

        plt.plot(times_snapshots[N_particles_per_side_i],
                 radial_momenta[N_particles_per_side_i] / (100 * M_solar * 11 * 1e5),
                 label="3D: $\Delta x = $ {:.1f} pc".format(400 / N_particles_per_side_i),
                 )


    sns.rugplot(SN_times_3D, color="k", linewidth=3)

    plt.xlabel(r"$t$ $[\mathrm{Myr}]$")
    plt.ylabel(r"$p / (100$ $M_\odot$ $N_\mathrm{SNe})$ $[\mathrm{km}$ $\mathrm{s}^{-1}]$")


    plt.xlim(xmax=100)
    plt.ylim(ymin=300,
             ymax=3000,
            )

    plt.yscale("log")

    plt.legend(loc="best")

In [ ]:
SN_times_3D